In [1]:
# Import
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from time import sleep

In [2]:
# FUNCTIONS

# Scrape
def scrape(url):
    # Initialize Browser
    options = Options()
    options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
    options.headless = True
    firefox_binary = FirefoxBinary()
    browser = webdriver.Firefox(executable_path=r'C:\bin\geckodriver.exe', options=options, firefox_binary=firefox_binary)
    browser.get(url)
    html = browser.page_source
    # Get Soup
    soup = bs(html, 'html.parser')
    return soup
    browser.close()

def table_todf(data, selector):
    table_data = data.select(selector)[0]
    df = pd.read_html(table_data.prettify(), flavor='bs4')[0]
    #ADD SOMETHING TO CHECK FOR/APPEND HYPERLINK
    return df

def percentiles(df):
    percentiles = []
    percentile_50 = float("{:.3f}".format(df.quantile(0.5)))
    percentiles.append(percentile_50)
    percentile_62 = float("{:.3f}".format(df.quantile(0.62)))
    percentiles.append(percentile_62)
    percentile_80 = float("{:.3f}".format(df.quantile(0.8)))
    percentiles.append(percentile_80)
    percentile_87 = float("{:.3f}".format(df.quantile(0.87)))
    percentiles.append(percentile_87)
    percentile_93 = float("{:.3f}".format(df.quantile(0.93)))
    percentiles.append(percentile_93)
    percentile_99 = float("{:.3f}".format(df.quantile(0.99)))
    percentiles.append(percentile_99)
    return percentiles

def reverse_percentiles(df):
    percentiles = []
    percentile_50 = float("{:.3f}".format(df.quantile(0.5)))
    percentiles.append(percentile_50)
    percentile_38 = float("{:.3f}".format(df.quantile(0.38)))
    percentiles.append(percentile_38)
    percentile_20 = float("{:.3f}".format(df.quantile(0.2)))
    percentiles.append(percentile_20)
    percentile_13 = float("{:.3f}".format(df.quantile(0.13)))
    percentiles.append(percentile_13)
    percentile_7 = float("{:.3f}".format(df.quantile(0.07)))
    percentiles.append(percentile_7)
    percentile_1 = float("{:.3f}".format(df.quantile(0.01)))
    percentiles.append(percentile_1)
    return percentiles

def sp_or_rp(row):
    ip = row['IP']
    g = row['G']
    ip_per_g = ip/g
    if ip_per_g >= 3:
        return "Starter"
    else:
        return "Reliever"
    
def milb_url(row):
    name = row['Name'].replace(' ', '-').replace('.', '')
    playerid = row['Link'].split('=')[1]
    return f'https://www.fangraphs.com/players/{name}/{playerid}'

In [3]:
# ### MLB Pitchers and DP Modifiers ###

# # Pitching Modifiers
# pitching_modifiers = {'$': [0.75, 1.50, 2.25, 3.00, 3.75, 4.50]}
# url = "https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&type=0&season=2023&season1=2023&ind=0&team=0&pageitems=2000000000&qual=0"
# selector = '#content > div.leaders-major_leaders-major__table__BLZyw > div.fg-data-grid.table-type > div.table-wrapper-outer > div > div.table-scroll > table'
# data = scrape(url)
# html = data.select(selector)[0]
# stats_table = table_todf(data, selector)

# links = html.findAll('a', href=True)
# player_links = []
# pitching_stats = pd.DataFrame()
# for link in links:
#     if("players" in link['href']):
#         player_links.append(link['href'])
        
# # Pitching Stats
# pitching_stats['Name'] = stats_table['Name']
# pitching_stats['Link'] = player_links
# pitching_stats['G'] = stats_table['G  G - Games Pitched']
# pitching_stats['IP'] = stats_table['IP  IP - Innings Pitched']
# pitching_stats['SO'] = stats_table['SO  SO - Strikeouts']
# pitching_stats['W'] = stats_table['W  W - Wins']
# pitching_stats['SV'] = stats_table['SV  SV - Saves']
# pitching_stats['HLD'] = stats_table['HLD  HLD - Holds']
# pitching_stats['ERA'] = stats_table['ERA  ERA - Earned Run Average ((ER*9)/IP)']
# pitching_stats['WHIP'] = ((stats_table['H  H - Hits Allowed'] + stats_table['BB  BB - Walks']) / stats_table['IP  IP - Innings Pitched'])
# pitching_stats['sp_or_rp'] = pitching_stats.apply (lambda row: sp_or_rp(row), axis=1)

# pitching_data = pitching_stats[pitching_stats['IP'] >= 50].copy() #Qualified pitchers

# dp_sp = pitching_data.loc[pitching_data['sp_or_rp'] == 'Starter'].copy()
# dp_rp = pitching_data.loc[pitching_data['sp_or_rp'] == 'Reliever'].copy()

# dp_sp_pitching_stat_columns = ['SO','W','ERA','WHIP']
# dp_rp_pitching_stat_columns = ['SO','SV','HLD','ERA','WHIP']

# for stat in dp_sp_pitching_stat_columns:
#     if stat in ['SO', 'W']:
#         pitching_modifiers[f'SP_{stat}'] = percentiles(dp_sp[stat])
#     else:
#         pitching_modifiers[f'SP_{stat}'] = reverse_percentiles(dp_sp[stat])

# for stat in dp_rp_pitching_stat_columns:
#     if stat in ['SO', 'SV', 'HLD']:
#         pitching_modifiers[f'RP_{stat}'] = percentiles(dp_rp[stat])
#     else:
#         pitching_modifiers[f'RP_{stat}'] = reverse_percentiles(dp_rp[stat])
        
# # Pitching DP Table
# p_modifiers_df = pd.DataFrame.from_dict(pitching_modifiers)
# p_modifiers_df['SP_SO'] = p_modifiers_df['SP_SO'].map('{:,.0f}'.format)
# p_modifiers_df['SP_W'] = p_modifiers_df['SP_W'].map('{:,.0f}'.format)
# p_modifiers_df['SP_ERA'] = p_modifiers_df['SP_ERA'].map('{:,.2f}'.format)
# p_modifiers_df['SP_WHIP'] = p_modifiers_df['SP_WHIP'].map('{:,.2f}'.format)
# p_modifiers_df['RP_SO'] = p_modifiers_df['RP_SO'].map('{:,.0f}'.format)
# p_modifiers_df['RP_SV'] = p_modifiers_df['RP_SV'].map('{:,.0f}'.format)
# p_modifiers_df['RP_HLD'] = p_modifiers_df['RP_HLD'].map('{:,.0f}'.format)
# p_modifiers_df['RP_ERA'] = p_modifiers_df['RP_ERA'].map('{:,.2f}'.format)
# p_modifiers_df['RP_WHIP'] = p_modifiers_df['RP_WHIP'].map('{:,.2f}'.format)

# pitching_stats.to_csv(f'output files/appeared_pitchers.csv', index=False, header=True)
# dp_pitchers.to_csv(f'output files/dp_pitchers.csv', index=False, header=True)

# p_modifiers_df

In [4]:
# ### MLB Hitters and DP Modifiers ###

# # Hitting Modifiers
# hitting_modifiers = {'$': [0.50, 1.00, 1.50, 2.00, 2.50, 3.00]}
# url = "https://www.fangraphs.com/leaders/major-league?pos=all&stats=bat&lg=all&type=0&season=2023&month=0&season1=2023&ind=0&team=0&pageitems=2000000000&qual=1&sortcol=0&sortdir=asc&pagenum=1"
# selector = '#content > div.leaders-major_leaders-major__table__BLZyw > div.fg-data-grid.table-type > div.table-wrapper-outer > div > div.table-scroll > table'
# data = scrape(url)
# html = data.select(selector)[0]
# stats_table = table_todf(data, selector)

# links = html.findAll('a', href=True)
# player_links = []
# hitting_stats = pd.DataFrame()
# for link in links:
#     if("players" in link['href']):
#         player_links.append(link['href'])
        
# # Hitting Stats
# hitting_stats['Name'] = stats_table['Name']
# hitting_stats['Link'] = player_links
# hitting_stats['AB'] = stats_table['AB  AB - At Bats']
# hitting_stats['R'] = stats_table['R  R - Runs']
# hitting_stats['HR'] = stats_table['HR  HR - Home Runs']
# hitting_stats['RBI'] = stats_table['RBI  RBI - Runs Batted In']
# hitting_stats['SB'] = stats_table['SB  SB - Stolen Bases']
# hitting_stats['AVG'] = stats_table['H  H - Hits'] / stats_table['AB  AB - At Bats']
# hitting_stats['OPS'] = (stats_table['H  H - Hits'] + stats_table['BB  BB - Walks'] + stats_table['HBP  HBP - Hit by Pitch']) / (stats_table['AB  AB - At Bats'] + stats_table['BB  BB - Walks'] + stats_table['IBB  IBB - Intentional Walks'] + stats_table['HBP  HBP - Hit by Pitch'] + stats_table['SF  SF - Sacrifice Fly']) + (stats_table['1B  1B - Singles'] + (2 * stats_table['2B  2B - Doubles']) + (3 * stats_table['3B  3B - Triples']) + (4 * stats_table['HR  HR - Home Runs'])) / stats_table['AB  AB - At Bats']

# dp_hitter_stat_columns = ['R','HR','RBI','SB','AVG','OPS']
# dp_hitters = hitting_stats[hitting_stats['AB'] >= 150].copy() #Qualified hitters

# for stat in dp_hitter_stat_columns:
#     hitting_modifiers[stat] = percentiles(dp_hitters[stat])

# # Hitting DP Table
# h_modifiers_df = pd.DataFrame.from_dict(hitting_modifiers)
# h_modifiers_df['R'] = h_modifiers_df['R'].map('{:,.0f}'.format)
# h_modifiers_df['HR'] = h_modifiers_df['HR'].map('{:,.0f}'.format)
# h_modifiers_df['RBI'] = h_modifiers_df['RBI'].map('{:,.0f}'.format)
# h_modifiers_df['SB'] = h_modifiers_df['SB'].map('{:,.0f}'.format)

# hitting_stats.to_csv(f'output files/appeared_hitters.csv', index=False, header=True)
# dp_hitters.to_csv(f'output files/dp_hitters.csv', index=False, header=True)

# h_modifiers_df

In [5]:
# Milb Pitchers
innings_qualifier = 10
url = f'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=2,4,5,6,7,8,9,10,11,14,12,13,15,16,17,18,30,32,33&stats=pit&qual={innings_qualifier}&type=0&team=&season=2023&seasonEnd=2023&org=&ind=0&splitTeam=false&players=&sort=25,1&pageitems=2000000000&pg=0'
selector = '#root-the-board > div.leaders-minor-league-data > div > div > div.table-wrapper-outer > div > div.table-scroll > table'
data = scrape(url)
html = data.select(selector)[0]
stats_table = table_todf(data, selector)

links = html.findAll('a', href=True)
player_links = []
milb_pitchers = pd.DataFrame()
for link in links:
    if("playerid" in link['href']):
        player_links.append(link['href'])
        
# Pitching Stats
milb_pitchers['Name'] = stats_table['Name']
milb_pitchers['Link'] = player_links
milb_pitchers['Link'] = milb_pitchers.apply(lambda row: milb_url(row), axis=1)

milb_pitchers.to_csv(f'output files/milb_pitchers.csv', index=False, header=True)

milb_pitchers

,Name,Link
0,Drew Thorpe,https://www.fangraphs.com/players/Drew-Thorpe/...
1,Tobias Myers,https://www.fangraphs.com/players/Tobias-Myers...
2,Cade Povich,https://www.fangraphs.com/players/Cade-Povich/...
3,Robert Gasser,https://www.fangraphs.com/players/Robert-Gasse...
4,Christian Roa,https://www.fangraphs.com/players/Christian-Ro...
...,...,...
3983,Johan Daniel,https://www.fangraphs.com/players/Johan-Daniel...
3984,Marco Gonzales,https://www.fangraphs.com/players/Marco-Gonzal...
3985,Manuel Figuereo,https://www.fangraphs.com/players/Manuel-Figue...
3986,Jhon Cabral,https://www.fangraphs.com/players/Jhon-Cabral/...


In [6]:
# Milb Hitters
ab_qualifier = 10
url = f'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=2,4,5,6,7,8,9,10,11,14,12,13,15,16,17,18,30,32,33&stats=bat&qual={ab_qualifier}&type=0&team=&season=2023&seasonEnd=2023&org=&ind=0&splitTeam=false&players=&sort=23,1&pageitems=2000000000&pg=0'
selector = '#root-the-board > div.leaders-minor-league-data > div > div > div.table-wrapper-outer > div > div.table-scroll > table'
data = scrape(url)
html = data.select(selector)[0]
stats_table = table_todf(data, selector)

links = html.findAll('a', href=True)
player_links = []
milb_hitters = pd.DataFrame()
for link in links:
    if("playerid" in link['href']):
        player_links.append(link['href'])
        
# Pitching Stats
milb_hitters['Name'] = stats_table['Name']
milb_hitters['Link'] = player_links
milb_hitters['Link'] = milb_hitters.apply(lambda row: milb_url(row), axis=1)

milb_hitters.to_csv(f'output files/milb_hitters.csv', index=False, header=True)
milb_hitters

,Name,Link
0,Ross Friedrick,https://www.fangraphs.com/players/Ross-Friedri...
1,Juan Soto,https://www.fangraphs.com/players/Juan-Soto/20123
2,Nolan Arenado,https://www.fangraphs.com/players/Nolan-Arenad...
3,Anthony Rendon,https://www.fangraphs.com/players/Anthony-Rend...
4,Trea Turner,https://www.fangraphs.com/players/Trea-Turner/...
...,...,...
3922,Steven Williams,https://www.fangraphs.com/players/Steven-Willi...
3923,Edgardo Lopez,https://www.fangraphs.com/players/Edgardo-Lope...
3924,Duncan Hewitt,https://www.fangraphs.com/players/Duncan-Hewit...
3925,Joseph Tavares,https://www.fangraphs.com/players/Joseph-Tavar...


In [7]:
milb_pitchers

,Name,Link
0,Drew Thorpe,https://www.fangraphs.com/players/Drew-Thorpe/...
1,Tobias Myers,https://www.fangraphs.com/players/Tobias-Myers...
2,Cade Povich,https://www.fangraphs.com/players/Cade-Povich/...
3,Robert Gasser,https://www.fangraphs.com/players/Robert-Gasse...
4,Christian Roa,https://www.fangraphs.com/players/Christian-Ro...
...,...,...
3983,Johan Daniel,https://www.fangraphs.com/players/Johan-Daniel...
3984,Marco Gonzales,https://www.fangraphs.com/players/Marco-Gonzal...
3985,Manuel Figuereo,https://www.fangraphs.com/players/Manuel-Figue...
3986,Jhon Cabral,https://www.fangraphs.com/players/Jhon-Cabral/...
